## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,72.52,100,1.21,overcast clouds
1,1,Ormara,PK,25.2088,64.6357,82.56,100,9.75,overcast clouds
2,2,Iskateley,RU,67.6803,53.1512,54.50,100,12.26,light rain
3,3,Uthal,PK,25.8072,66.6219,83.50,22,8.93,few clouds
4,4,New Norfolk,AU,-42.7826,147.0587,49.91,100,2.77,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Cloudiness,Wind Speed,Current Description
1,1,Ormara,PK,25.2088,64.6357,82.56,100,9.75,overcast clouds
3,3,Uthal,PK,25.8072,66.6219,83.50,22,8.93,few clouds
8,8,Lancaster,US,40.2504,-76.2499,76.03,0,8.99,clear sky
17,17,Nishihara,JP,26.1842,127.7558,80.26,75,16.11,light rain
23,23,Aden,YE,12.7794,45.0367,85.98,96,2.84,overcast clouds
25,25,Jamestown,US,42.0970,-79.2353,76.71,0,8.01,clear sky
26,26,Juneau,US,58.3019,-134.4197,75.20,20,11.50,few clouds
29,29,Hilo,US,19.7297,-155.0900,82.13,100,9.22,overcast clouds
30,30,Sola,VU,-13.8833,167.5500,81.34,74,16.67,broken clouds
31,31,Vaini,TO,-21.2000,-175.2000,80.76,40,11.50,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                252
City                   252
Country                252
Lat                    252
Lng                    252
Max Temp               252
Cloudiness             252
Wind Speed             252
Current Description    252
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
full_cities_df= preferred_cities_df.dropna()
full_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Cloudiness,Wind Speed,Current Description
1,1,Ormara,PK,25.2088,64.6357,82.56,100,9.75,overcast clouds
3,3,Uthal,PK,25.8072,66.6219,83.50,22,8.93,few clouds
8,8,Lancaster,US,40.2504,-76.2499,76.03,0,8.99,clear sky
17,17,Nishihara,JP,26.1842,127.7558,80.26,75,16.11,light rain
23,23,Aden,YE,12.7794,45.0367,85.98,96,2.84,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng", "Current Description"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
1,Ormara,PK,82.56,25.2088,64.6357,overcast clouds,
3,Uthal,PK,83.50,25.8072,66.6219,few clouds,
8,Lancaster,US,76.03,40.2504,-76.2499,clear sky,
17,Nishihara,JP,80.26,26.1842,127.7558,light rain,
23,Aden,YE,85.98,12.7794,45.0367,overcast clouds,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
1,Ormara,PK,82.56,25.2088,64.6357,overcast clouds,Kenara Colony House for Rent
3,Uthal,PK,83.50,25.8072,66.6219,few clouds,Al-Mustafa Hotel
8,Lancaster,US,76.03,40.2504,-76.2499,clear sky,Refreshing Mountain Retreat and Adventure Center
17,Nishihara,JP,80.26,26.1842,127.7558,light rain,Minshuku Agaihama
23,Aden,YE,85.98,12.7794,45.0367,overcast clouds,Sama Emirate Hotel
25,Jamestown,US,76.71,42.0970,-79.2353,clear sky,Hampton Inn & Suites Jamestown
26,Juneau,US,75.20,58.3019,-134.4197,few clouds,"Baranof Downtown, BW Signature Collection"
29,Hilo,US,82.13,19.7297,-155.0900,overcast clouds,Hilo Hawaiian Hotel
30,Sola,VU,81.34,-13.8833,167.5500,broken clouds,Leumerus Bungalows
31,Vaini,TO,80.76,-21.2000,-175.2000,scattered clouds,Keleti Beach Resort


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/Vacation_Search.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))